In [19]:
import pandas as pd
from io import StringIO
import numpy as np
import os
import re
import datetime

In [20]:
root_dir_metric = "data_metric - Copy"

filename_person = "1-AMPerson1D1-live_EPOCX_223759_2024.06.05T10.53.48+08.00.md.mc.pm.fe.bp.csv"
filename_metric = "Data Set EEG ONLY v2 (Day 1 - Attention) - Sheet1.csv"
data_metric = pd.read_csv(f"{root_dir_metric}/{filename_metric}")

# Step 1: Extract the column name from the filename using regex
column_name_match = re.search(r'(\d+-\w+)', filename_person)
if column_name_match:
    column_name = column_name_match.group(1)
    print(column_name)
else:
    raise ValueError("Column name not found in the filename")

# Transpose the DataFrame
transposed_data_metric = data_metric.transpose()

# Set the second row (index 1) as the header
new_header = transposed_data_metric.iloc[1]  # second row as header
transposed_data_metric = transposed_data_metric[2:]  # data starts from the third row
transposed_data_metric.columns = new_header  # set the header

# Reset index to default integer indices
transposed_data_metric.reset_index(drop=True, inplace=True)

# Drop rows before index 19 (including rows 0 to 18)
transposed_data_metric = transposed_data_metric.drop(transposed_data_metric.index[:19])


# Rename columns based on their positional indices or current names
transposed_data_metric.columns = [ "Online/Live", "Min/Max","Timestamp"] + list(transposed_data_metric.columns[3:])

if "Day 1" in filename_metric:
    transposed_data_metric["Timestamp"] = pd.to_datetime("June 5, 2024 ") + pd.to_timedelta(transposed_data_metric["Timestamp"])
if "Day 2" in filename_metric:
    transposed_data_metric["Timestamp"] = pd.to_datetime("June 6, 2024 ") + pd.to_timedelta(transposed_data_metric["Timestamp"])
if "Day 3" in filename_metric:
    transposed_data_metric["Timestamp"] = pd.to_datetime("June 7, 2024") + pd.to_timedelta(transposed_data_metric["Timestamp"])

transposed_data_metric

1-AMPerson1D1


In [21]:
transposed_data_metric

,Online/Live,Min/Max,Timestamp,1-AMPerson1D1,1-PMPerson1D1,10-AMPerson10D1,10-PMPerson10D1,2-AMPerson2D1,2-PMPerson2D1,3-AMPerson3D1,...,5-AMPerson5D1,5-PMPerson5D1,6-AMPerson6D1,6-PMPerson6D1,7-AMPerson7D1,7-PMPerson7D1,8-AMPerson8D1,8-PMPerson8D1,9-AMPerson9D1,9-PMPerson9D1
19,Online,Min,2024-06-05 09:41:54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-14.714978,NaN,NaN,NaN
20,Online,Max,2024-06-05 09:41:54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.508995,NaN,NaN,NaN
21,Online,Min,2024-06-05 09:41:58,NaN,NaN,-12.613671,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,Online,Max,2024-06-05 09:41:58,NaN,NaN,-5.735361,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,Online,Min,2024-06-05 09:42:01,NaN,NaN,NaN,NaN,NaN,NaN,-18.081623,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7351,Live,Max,2024-06-05 15:39:05,NaN,-4.910878,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7352,Live,Min,2024-06-05 15:39:19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-11.017278
7353,Live,Max,2024-06-05 15:39:19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.698471
7354,Live,Min,2024-06-05 15:39:39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-11.006492


In [27]:
def find_non_matching_indices(array_person, array_metric):
    person_matched = True
    metric_matched = True

    for person_item in array_person:
        person_string = person_item[0]
        person_value = float(person_item[1])  # Convert to float for numerical comparison
        found_match = False
        
        for metric_item in array_metric:
            metric_string = metric_item[0]
            metric_value = metric_item[1]
            
            if person_string == metric_string and person_value == metric_value:
                found_match = True
                break
        
        if not found_match:
            person_matched = False
            print(f"No matching value found for person item '{person_item}'.")

    for metric_item in array_metric:
        metric_string = metric_item[0]
        metric_value = metric_item[1]
        found_match = False
        
        for person_item in array_person:
            person_string = person_item[0]
            person_value = float(person_item[1])  # Convert to float for numerical comparison
            
            if metric_string == person_string and metric_value == person_value:
                found_match = True
                break
        
        if not found_match:
            metric_matched = False
            print(f"No matching value found for metric item '{metric_item}'.")

    if person_matched and metric_matched:
        print("Everything is matched.")

# Example usage
array_person = [['1717551714', '10.508995'],
                ['1717551714', '-14.714978'],
                ['1717551718', '-5.735361']]

array_metric = [['1717551714', -14.714978],
                ['1717551714', 10.508995],
                ['1717551718', -12.613671]]

find_non_matching_indices(array_person, array_metric)


No matching value found for person item '['1717551718', '-5.735361']'.
No matching value found for metric item '['1717551718', -12.613671]'.
